<a href="https://colab.research.google.com/github/diogopaz/projeto-marvel/blob/DatabaseBrch/projeto_marvel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importando bibliotecas necessárias

In [ ]:
!pip install dotenv

In [ ]:
import requests
import hashlib
import time
import os
import pandas as pd
from dotenv import load_dotenv
import sqlite3
from google.colab import userdata


# Carregando variáveis de ambiente
Estamos fazendo upload do arquivo .env com as chaves de acesso à API no ambeinte de execução do colab.

In [ ]:
load_dotenv(dotenv_path='/content/.env')
public_key = userdata.get('MARVEL_PUBLIC_KEY') or os.getenv('MARVEL_PUBLIC_KEY')
private_key = userdata.get('MARVEL_PRIVATE_KEY') or os.getenv('MARVEL_PRIVATE_KEY')

# Montando os parâmetros para realizar a requisição

In [ ]:
ts = str(time.time())
to_hash = ts + str(private_key) + str(public_key)
hash_md5 = hashlib.md5(to_hash.encode('utf-8')).hexdigest()

params = {
    'apikey': public_key,
    'ts': ts,
    'hash': hash_md5,
    'limit': 100
}

# Realizando a requisição

In [ ]:
url = 'https://gateway.marvel.com/v1/public/characters'

response = requests.get(url, params=params)
data = response.json()
data

In [ ]:
df = pd.DataFrame(data['data']['results'])
df.columns

# Criação da tabela de personagens no banco de dados sqlite

In [ ]:
def get_auth_params():
    ts = str(time.time())
    to_hash = ts + private_key + public_key
    hash_md5 = hashlib.md5(to_hash.encode()).hexdigest()
    return {
        'ts': ts,
        'apikey': public_key,
        'hash': hash_md5
    }

In [ ]:
url = 'https://gateway.marvel.com/v1/public/characters'
# >>> Conecta (ou cria) o banco SQLite
conn = sqlite3.connect('marvel.db')
cursor = conn.cursor()

# >>> Cria tabela characters (sem thumbnail/resource_uri)
cursor.execute('''
CREATE TABLE IF NOT EXISTS characters (
    id INTEGER PRIMARY KEY,
    name TEXT,
    description TEXT,
    modified TEXT
)
''')

# >>> Cria tabela character_comics (relacionamento N:N entre personagens e comics)
cursor.execute('''
CREATE TABLE IF NOT EXISTS character_comics (
    character_id INTEGER,
    comic_id INTEGER,
    PRIMARY KEY (character_id, comic_id)
)
''')

conn.commit()

# >>> Função para montar hash exigido pela Marvel
def get_auth_params():
    ts = str(time.time())
    to_hash = ts + private_key + public_key
    hash_md5 = hashlib.md5(to_hash.encode()).hexdigest()
    return {
        'ts': ts,
        'apikey': public_key,
        'hash': hash_md5
    }

# >>> Função para salvar um personagem no banco
def save_character(char):
    cursor.execute('''
    INSERT OR REPLACE INTO characters (id, name, description, modified)
    VALUES (?, ?, ?, ?)
    ''', (
        char['id'],
        char['name'],
        char.get('description', ''),
        char.get('modified', '')
    ))

    # >>> Salva os comics vinculados ao personagem
    for comic in char.get('comics', {}).get('items', []):
        resource_uri = comic.get('resourceURI', '')
        try:
            comic_id = int(resource_uri.strip().split('/')[-1])
            cursor.execute('''
                INSERT OR IGNORE INTO character_comics (character_id, comic_id)
                VALUES (?, ?)
            ''', (char['id'], comic_id))
        except (IndexError, ValueError):
            print(f'❌ Erro ao extrair comic_id de {resource_uri}')

    conn.commit()

# >>> Baixa e salva todos os personagens
def fetch_and_save_characters():
    limit = 100
    offset = 0
    total = 1  # valor fictício inicial

    while offset < total:
        print(f'⏳ Buscando personagens... offset={offset}')
        params = get_auth_params()
        params.update({
            'limit': limit,
            'offset': offset
        })

        response = requests.get(url, params=params)
        if response.status_code != 200:
            print(f'⚠️ Erro na requisição: {response.status_code}')
            break

        data = response.json()['data']
        total = data['total']
        results = data['results']

        for char in results:
            save_character(char)

        offset += limit
        time.sleep(0.1)  # respeita limites

    print('✅ Todos os personagens e comics foram salvos no banco!')

# >>> Executa
fetch_and_save_characters()

# >>> Fecha conexão
conn.close()



In [ ]:
conn = sqlite3.connect('marvel.db')
cursor = conn.cursor()
cursor.execute('select * from characters')
results_sqlite = cursor.fetchall()
conn.close()
results_sqlite

In [ ]:
try:
    conn.commit()
except:
    pass

try:
    conn.close()
except:
    pass

conn = sqlite3.connect('marvel.db')
cursor = conn.cursor()

# Tabela de comics
cursor.execute('''
CREATE TABLE IF NOT EXISTS comics (
    id INTEGER PRIMARY KEY,
    title TEXT,
    page_count INTEGER,
    variant_count INTEGER
)
''')

# Relacionamento comic -> creator
cursor.execute('''
CREATE TABLE IF NOT EXISTS comic_creators (
    comic_id INTEGER,
    creator_id INTEGER,
    role TEXT,
    PRIMARY KEY (comic_id, creator_id, role)
)
''')

# Preços das comics
cursor.execute('''
CREATE TABLE IF NOT EXISTS comic_prices (
    comic_id INTEGER,
    type TEXT,
    price REAL,
    PRIMARY KEY (comic_id, type)
)
''')

conn.commit()



In [ ]:
def save_comic(comic):
    comic_id = comic.get('id')
    if not comic_id or 'title' not in comic:
        print(f"⚠️ Comic ignorada por dados incompletos: {comic_id}")
        return

    title = comic['title']
    page_count = comic.get('pageCount', 0)
    variant_count = len(comic.get('variants', []))

    # Salva a comic
    cursor.execute('''
    INSERT OR REPLACE INTO comics (id, title, page_count, variant_count)
    VALUES (?, ?, ?, ?)
    ''', (comic_id, title, page_count, variant_count))

    # Salva os preços
    for price in comic.get('prices', []):
        cursor.execute('''
        INSERT OR REPLACE INTO comic_prices (comic_id, type, price)
        VALUES (?, ?, ?)
        ''', (comic_id, price.get('type', ''), price.get('price', 0.0)))

    # Salva os creators associados
    for creator in comic.get('creators', {}).get('items', []):
        try:
            resource_uri = creator.get('resourceURI', '')
            creator_id = int(resource_uri.strip().split('/')[-1])
            role = creator.get('role', '')
            cursor.execute('''
            INSERT OR IGNORE INTO comic_creators (comic_id, creator_id, role)
            VALUES (?, ?, ?)
            ''', (comic_id, creator_id, role))
        except (IndexError, ValueError):
            print(f'❌ Erro ao extrair creator_id de {resource_uri}')

    conn.commit()


In [ ]:
def fetch_and_save_comics():
    limit = 100
    offset = 0
    total = 1  # valor inicial fictício
    comics_url = "https://gateway.marvel.com/v1/public/comics"  # ou use variável se já tiver `url_comics`

    while offset < total:
        print(f'⏳ Buscando comics... offset={offset}')
        params = get_auth_params()
        params.update({
            'limit': limit,
            'offset': offset
        })

        response = requests.get(comics_url, params=params)
        if response.status_code != 200:
            print(f'⚠️ Erro na requisição: {response.status_code}: {response.text}')
            break

        data = response.json()['data']
        total = data['total']
        results = data['results']

        for comic in results:
            save_comic(comic)

        offset += limit
        time.sleep(0.1)

    print('✅ Todas as comics foram salvas no banco!')


In [ ]:
fetch_and_save_comics()